<a href="https://colab.research.google.com/github/vigilant-umbrella/hcv-prediction/blob/main/hcv_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [ ]:
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Data Preprocessing

In [ ]:
%%shell
if ! [ -f "hcvdat0.csv" ]; then
    wget https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv 
fi

--2021-09-26 12:50:42--  https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46183 (45K) [application/x-httpd-php]
Saving to: ‘hcvdat0.csv’

hcvdat0.csv         100%[===================>]  45.10K   174KB/s    in 0.3s    

2021-09-26 12:50:44 (174 KB/s) - ‘hcvdat0.csv’ saved [46183/46183]



In [ ]:
data = pd.read_csv('hcvdat0.csv')

data = data[(data['Category']!='0=Blood Donor') & (data['Category']!='0s=suspect Blood Donor')].reset_index(drop=True)

X = data.drop(['Unnamed: 0', 'Category'], axis=1)
category = pd.DataFrame(data['Category'])

X = pd.concat([X, pd.get_dummies(X.Sex, drop_first=True)], axis=1)
X = X.drop(['Sex'], axis=1)

X['ALB'] = X['ALB'].fillna(X['ALB'].median())
X['ALP'] = X['ALP'].fillna(X['ALP'].median())
X['ALT'] = X['ALT'].fillna(X['ALT'].median())
X['CHOL'] = X['CHOL'].fillna(X['CHOL'].mean())
X['PROT'] = X['PROT'].fillna(X['PROT'].mean())

enc = OrdinalEncoder()
y = pd.DataFrame(enc.fit_transform(pd.DataFrame(category)), columns=['category'])

In [ ]:
cols_to_drop = ['m', 'GGT', 'AST', 'CREA', 'PROT']

In [ ]:
orig_scaler = MinMaxScaler()
scaled_orig_X = orig_scaler.fit_transform(X)

X_dropped = X.drop(cols_to_drop, axis=1)
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X_dropped)

In [ ]:
result = []

# Utils

In [ ]:
def generate_kfold_report(results):
    report = {}
    report['Accuracy'] = sum(results['test_accuracy'])/len(results['test_accuracy'])
    report['Precision'] = sum(results['test_precision_macro'])/len(results['test_precision_macro'])
    report['Recall'] = sum(results['test_recall_macro'])/len(results['test_recall_macro'])
    report['F1 Score'] = sum(results['test_f1_macro'])/len(results['test_f1_macro'])

    return report

# LogisticRegresssion

## Without Removing Features

In [ ]:
lr = LogisticRegression(
    C=1.25,
    fit_intercept=True,
    l1_ratio=0.5,
    max_iter=50,
    penalty='elasticnet',
    random_state=221,
    solver='saga'
    )

lrcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

lrcv_results = cross_validate(
    lr,
    scaled_orig_X,
    y['category'],
    cv=lrcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(lrcv_results)

## Removing Features

In [ ]:
lr = LogisticRegression(
    C=1.25,
    fit_intercept=True,
    l1_ratio=0.5,
    max_iter=50,
    penalty='elasticnet',
    random_state=221,
    solver='saga'
    )

lrcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

lrcv_results = cross_validate(
    lr,
    scaled_X,
    y['category'],
    cv=lrcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(lrcv_results)

In [ ]:
result.append(
    {
        'model': 'Logistic Regression',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# KNeighborsClassifier

## Without Removing Features

In [ ]:
knclf = KNeighborsClassifier(
    n_neighbors=10,
    algorithm='ball_tree',
    p=4
    )

kncv = StratifiedKFold(n_splits=10, shuffle=True, random_state=182)

kncv_results = cross_validate(
    knclf,
    scaled_orig_X,
    y['category'],
    cv=kncv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(kncv_results)

## Removing Features

In [ ]:
knclf = KNeighborsClassifier(
    n_neighbors=10,
    algorithm='ball_tree',
    p=4
    )

kncv = StratifiedKFold(n_splits=10, shuffle=True, random_state=182)

kncv_results = cross_validate(
    knclf,
    scaled_X,
    y['category'],
    cv=kncv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(kncv_results)

In [ ]:
result.append(
    {
        'model': 'k-nearest neighbors',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# GaussianNB

## Without Removing Features

In [ ]:
gnb = GaussianNB(var_smoothing=1e-10)

gnbcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=302)

gnbcv_results = cross_validate(
    gnb,
    scaled_orig_X,
    y['category'],
    cv=gnbcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(gnbcv_results)

## Removing Features

In [ ]:
gnb = GaussianNB(var_smoothing=1e-10)

gnbcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=302)

gnbcv_results = cross_validate(
    gnb,
    scaled_X,
    y['category'],
    cv=gnbcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(gnbcv_results)

In [ ]:
result.append(
    {
        'model': 'Gaussian naive Bayes',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# DecisionTreeClassifier

## Without Removing Features

In [ ]:
dtclf = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=7,
    max_features=None,
    min_samples_leaf=3,
    min_samples_split=0.4,
    random_state=559,
    splitter='random'
    )

dtcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=985)

dtcv_results = cross_validate(
    dtclf,
    scaled_orig_X,
    y['category'],
    cv=dtcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(dtcv_results)

## Removing Features

In [ ]:
dtclf = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=7,
    max_features=None,
    min_samples_leaf=3,
    min_samples_split=0.4,
    random_state=559,
    splitter='random'
    )

dtcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=985)

dtcv_results = cross_validate(
    dtclf,
    scaled_X,
    y['category'],
    cv=dtcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(dtcv_results)

In [ ]:
result.append(
    {
        'model': 'Decision Tree',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# RandomForestClassifier

## Without Removing Features

In [ ]:
rfclf = RandomForestClassifier(
    criterion='gini',
    max_features='sqrt',
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=50,
    random_state=67
    )

rfcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=309)

rfcv_results = cross_validate(
    rfclf,
    scaled_orig_X,
    y['category'],
    cv=rfcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(rfcv_results)

## Removing Features

In [ ]:
rfclf = RandomForestClassifier(
    criterion='gini',
    max_features='sqrt',
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=50,
    random_state=67
    )

rfcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=309)

rfcv_results = cross_validate(
    rfclf,
    scaled_X,
    y['category'],
    cv=rfcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(rfcv_results)

In [ ]:
result.append(
    {
        'model': 'Random Forest',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# SVC

## Without Removing Features

In [ ]:
svc = SVC(
    C=0.1,
    coef0=0.3,
    degree=2,
    gamma='scale',
    kernel='poly',
    random_state=98,
    shrinking=True
    )

svccv = StratifiedKFold(n_splits=10, shuffle=True, random_state=911)

svccv_results = cross_validate(
    svc,
    scaled_orig_X,
    y['category'],
    cv=svccv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(svccv_results)

## Removing Features

In [ ]:
svc = SVC(
    C=0.1,
    coef0=0.3,
    degree=2,
    gamma='scale',
    kernel='poly',
    random_state=98,
    shrinking=True
    )

svccv = StratifiedKFold(n_splits=10, shuffle=True, random_state=911)

svccv_results = cross_validate(
    svc,
    scaled_X,
    y['category'],
    cv=svccv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(svccv_results)

In [ ]:
result.append(
    {
        'model': 'Support Vector Machine',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# MLPClassifier

## Without Removing Features

In [ ]:
mlpclf = MLPClassifier(
    batch_size=8,
    activation='relu',
    early_stopping=False,
    hidden_layer_sizes=(32, 32, 32),
    max_iter=500,
    random_state=377,
    solver='lbfgs'
    )

mlpclfcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=496)

mlpclfcv_results = cross_validate(
    mlpclf,
    scaled_orig_X,
    y['category'],
    cv=mlpclfcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_without = generate_kfold_report(mlpclfcv_results)

## Removing Features

In [ ]:
mlpclf = MLPClassifier(
    batch_size=8,
    activation='relu',
    early_stopping=False,
    hidden_layer_sizes=(32, 32, 32),
    max_iter=500,
    random_state=377,
    solver='lbfgs'
    )

mlpclfcv = StratifiedKFold(n_splits=10, shuffle=True, random_state=496)

mlpclfcv_results = cross_validate(
    mlpclf,
    scaled_X,
    y['category'],
    cv=mlpclfcv,
    scoring=['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'],
    n_jobs=-1
    )

In [ ]:
report_with = generate_kfold_report(mlpclfcv_results)

In [ ]:
result.append(
    {
        'model': 'Mutli-layer Perceptron',
        'Without using described approach': report_without,
        'Using described approach': report_with
    }
)

# Evaluating Results

In [ ]:
res = pd.json_normalize(result)
res = res.set_index('model')
res.index.name = None
res.columns = res.columns.str.split('.', expand=True)
res = res.sort_values(by=('Using described approach', 'Accuracy'))
res

Without using described approach  ... Using described approach
                                               Accuracy  ...                 F1 Score
Decision Tree                                  0.571429  ...                 0.737143
Gaussian naive Bayes                           0.708929  ...                 0.772540
Random Forest                                  0.678571  ...                 0.777302
Logistic Regression                            0.692857  ...                 0.775238
k-nearest neighbors                            0.669643  ...                 0.807778
Support Vector Machine                         0.641071  ...                 0.788492
Mutli-layer Perceptron                         0.612500  ...                 0.840476

[7 rows x 8 columns]